
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan

------------------------------------------------------------

# Mesure de concentrations de gaz 

Les instruments de mesure de concentration gazeuses de la famille MQ  fournissent un signal analogique c'est pourquoi il faut les convertir en signal numérique pour pouvoir les exploiter avec une RaspberryPI

## Utilisation d'un convertisseur Analogique --> digital

Nous avons à notre disposition deux convertisseurs analogique numérique dont le fonctionnement est identique. Ce qui les diffférencie est la grandeur privilégiée: fréquence d'échantillonnage pour le premier, précision de l'échantillonnage pour le second.

 * [ADS1015](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1015.html) Résolution de 12 bit, Fréquence d'échantillonnage Min : 128 Hz Max :3300 Hz (SPS_128 à SPS_3300)
 * [ADS1115](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1115.html) Résolution  de 16 bit, Fréquence d'échantillonage Min : 8 Hz Max : 860 Hz (SPS_8 à SPS_860)

Ce qui  diffférencie les deux convertisseurs est la grandeur privilégiée:
* pour ADS1015 c'est la fréquence de l'échantillonnage 
* pour ADS1115 c'est la précision  de l'échantillonage.

La totalité des commandes sont communes sauf le réglage de la fréquence d'échantillonage, dans les hautes fréquences pour le `ADS1015`et dans les basses fréquences pour le `ADS1115`.

Deux modes de fonctionnement sont possibles :
 * en mode différentiel le signal est mesuré par différence entre la broche de sortie et la masse
 * en mode direct le signal est mesuré directement sur la broche de sortie

La figure ci-dessous présente l'ensemble du dispositif :

![Convertisseurs vue générale](images/GazAnalogique.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Le câblage du convertisseurs ne présente pas de difficulté particulière, le point requérant un peu d'attention est le choix de l'adresse qui sera envoyée sur le bus `I2C` car il faut choisir cette adresse parmi les 4 possibles :  `0x49` `0x48` `0x4B` `0x4A`. Le choix se fait en établissant une connexion entre la broche `ADDR` et l'une des 4 broches `VDD` `GND` `SCL` `SDA`. Le tableau ci-dessous indique les équivalences des connexions :

| Broche | Adresse |
|:------ | ------- |
| `VDD`  | `0x49`  |
| `GND`  | `0x48`  |
| `SCL`  | `0x4B`  |
| `SDA`  | `0x4A`  |

Le capteur analogique (ici un capteur de gaz) doit être alimenté en 5 Volts (cables rouges) et les retours des données sur le RaspberryPi en 3,3V (Cables oranges). Le cable bleu joignant les broches ADDR et VDD du convertisseur montre qu'il faudra prendre l'adresse `0x49` sur le bus I2C.

          
## Importation des librairies nécessaires à l'utilisation du convertisseur.
On n'importe pas de librairies propre au capteur car il s'agit d'un capteur `analogiques` qui transmet une variation de potentiel sur un fil série.

In [1]:
import mraa, upm
from time import sleep
from upm import pyupm_ads1x15 as upm

## Récupération des paramètres d'état de l'échantillonneur

On déclare ici l'échantillonneur et on récupère les valeurs de ses paramètres d'opération (gain, fréquence et mode). 
 * getGain()
 * getSPS()
 * getContinuous()
 
On fixe ensuite ces paramètres pour la manipulation que l'on souhaite effectuer.
 * setGain()
 * setSPS()
 * setContinuous()
 
Le paramètre SPS contrôle la fréquence d'échantillonnage.
* Pour ADS1015 les valeurs possibles sont : SPS_128, SPS_250, SPS_490, SPS_920, SPS_1600, SPS_2400, SPS_3300.

* Pour ADS1115 les valeurs possibles étant :  SPS_8, SPS_16, SPS_32, SPS_64,
  SPS_128, SPS_250, SPS_475, SPS_860. 
  
Le choix du gain est aussi un paramètre crucial. La valeur doit être choisie en fonction des propriétés du capteur :  `GAIN_TWOTHIRDS` = ADS1X15_PGA_6_144V, `GAIN_ONE` = ADS1X15_PGA_4_096V, `GAIN_TWO` = ADS1X15_PGA_2_048V, `GAIN_FOUR` = ADS1X15_PGA_1_024V, `GAIN_EIGHT` = ADS1X15_PGA_0_512V, `GAIN_SIXTEEN` = ADS1X15_PGA_0_256V. 

In [2]:
gaz = upm.ADS1015(0, 0x49)
gain = gaz.getGain()
rate = gaz.SPS_920
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))
gaz.setContinuous(True) # Pour ne pas faire de conversion en continu
gaz.setGain(upm.ADS1X15.GAIN_ONE) # Passage à un gain 1024
gaz.setSPS(rate) # Passage à une fréquence de 128 définie par la valeur de rate précédemment : SPS_128
# Vérification
gain = gaz.getGain()
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("\nNouvelles valeurs :")
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))

Gain :   512  SPS :   128 Continuous :     0

Nouvelles valeurs :
Gain :   512  SPS :    96 Continuous :     1


## Mesure directe des variables qualité alcool méthane

Comme les données du capteur sont envoyées sur la broche analogique A3 du convertisseur, la mesure se fait par la commande suivante:

In [3]:
# qual = gaz.getSample(upm.ADS1X15.SINGLE_0)
mesuregaz = gaz.getSample(upm.ADS1X15.SINGLE_1)
#mesuregaz_diff = gaz.getSample(upm.ADS1X15.DIFF_0_3)
# meth = gaz.getSample(upm.ADS1X15.DIFF_2_3)
print ("MesureGaz : {0:.2f}\n".format(mesuregaz))
#print ("MesureGaz : {0:.2f}\n".format(mesuregaz), "MesureGazDiff : {0:.2f}\n".format(mesuregaz_diff))

MesureGaz : 0.53



# série de mesures à l'aide d'une boucle
Dans le groupe suivant on répète 20 fois la mesure en partant de 0 et en allant jusqu'à 19 (inférieur à 20) au pas de 1 par défaut.

In [5]:
for num in range(0,2):
#    qual = gaz.getSample(upm.ADS1X15.SINGLE_0)
    mesuregaz = gaz.getSample(upm.ADS1X15.SINGLE_1)
#    meth = gaz.getSample(upm.ADS1X15.DIFF_2_3)
    print (num , "MesureGaz : {0:.2f}\n".format(mesuregaz))    
    sleep(2)

0 MesureGaz : 0.52

1 MesureGaz : 0.56

